In [ ]:
# pip install to request access to website
!pip3 install requests

In [ ]:
# pip install beautifulsoup4 to access the library
!pip install beautifulsoup4

In [ ]:
# imports for BeautifulSoup and other important libraries
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

In [ ]:
#GET FULL HTML OF PAGE
import requests
from bs4 import BeautifulSoup
r = requests.get('https://classiccars.com/newest-listings')

# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')


# lines for checking if it has the links but not that important
#print(r.content)
#print(soup.prettify())

In [ ]:
# Modles dictionary for all possible car brands that could be found in the website
# This will help recognize more cars and allow for more cars to be picked up
models = [
    "Alfa Romeo", "AMC", "Aston Martin", "Audi", "Austin-Healey", "Bentley", "BMW",
    "Buick", "Cadillac", "Chevrolet", "Chrysler", "Datsun", "Dodge", "Ferrari", "Fiat",
    "Ford", "GMC", "Harley-Davidson", "Honda", "Hudson", "Hummer", "International",
    "Jaguar", "Jeep", "Lamborghini", "Land Rover", "Lincoln", "Maserati", "Mazda",
    "Mercedes-Benz", "Mercury", "MG", "Mitsubishi", "Nash", "Nissan", "Oldsmobile",
    "Packard", "Plymouth", "Pontiac", "Porsche", "Rolls-Royce", "Shelby", "Studebaker",
    "Subaru", "Suzuki", "Toyota", "Triumph", "Volkswagen", "Volvo", "Willys"
]


In [ ]:

  model=models[1]
  page_url = f'https://classiccars.com/listings/find/all-years/{model}?p=1&ps=60'

  # Send an HTTP request to the current page
  r = requests.get(page_url)

  # Parsing the HTML
  soup = BeautifulSoup(r.content, 'html.parser')

  # Find the div containing the car listings
  s = soup.find('div', class_='b w100 mrg-b-lg text-center').findAll("span")

In [ ]:
int(s[2].get_text(strip=True).rstrip('.'))

2

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Create an empty set to store car links
urls = set()

# For loop that will go through multiple url links from the classic cars website
for model in models:
  print(model)

  r = requests.get(f'https://classiccars.com/listings/find/all-years/{model}?p=1&ps=60')
  soup = BeautifulSoup(r.content, 'html.parser')
  s = soup.find('div', class_='b w100 mrg-b-lg text-center').findAll("span")
  maxpg=int(s[2].get_text(strip=True).rstrip('.'))
  print(maxpg)
  # Iterate through 15 pages of car listings
  for page in range(1, maxpg):  # Pages 1 to 15
      # Construct the URL for the current page
      page_url = f'https://classiccars.com/listings/find/all-years/{model}?p={page}&ps=60'

      # Send an HTTP request to the current page
      r = requests.get(page_url)

      # Parsing the HTML
      soup = BeautifulSoup(r.content, 'html.parser')

      # Find the div containing the car listings
      s = soup.find('div', class_='fx-item fi-srrw-col-2 fx-va-top')
      content = s.find_all('a')

      # Extract and filter links on the current page
      for link in content:
          href = link.get('href')
          # Check if the link is a car listing (adjust the condition based on the actual structure)
          if href and '/listings/view/' in href:
              # Join the base URL with the relative link to get the absolute URL
              absolute_url = urljoin('https://classiccars.com', href)
              urls.add(absolute_url)

  # Print the collected links
  #for link in car_links:
      #print(link)


Alfa Romeo
3
AMC
2
Aston Martin
2
Audi
2
Austin-Healey
3
Bentley
4
BMW
10
Buick
13
Cadillac
18
Chevrolet
150
Chrysler
7
Datsun
3
Dodge
25
Ferrari
5
Fiat
3
Ford
131
GMC
12
Harley-Davidson
4
Honda
4
Hudson
2
Hummer
2
International
3
Jaguar
10
Jeep
14
Lamborghini
2
Land Rover
7
Lincoln
9
Maserati
2
Mazda
3
Mercedes-Benz
26
Mercury
8
MG
8
Mitsubishi
2
Nash
2
Nissan
6
Oldsmobile
13
Packard
3
Plymouth
14
Pontiac
25
Porsche
16
Rolls-Royce
5
Shelby
3
Studebaker
4
Subaru
2
Suzuki
2
Toyota
11
Triumph
4
Volkswagen
14
Volvo
2
Willys
3


In [ ]:
# Prints all car urls
urls

In [ ]:
# Will show how many car links there are in urls
len(urls)

34651

In [ ]:
# possibly betterv2
import requests
from bs4 import BeautifulSoup
import pandas as pd


# Initialize counters
successful_requests = 0  # To count successful requests
failed_requests = 0  # To count failed requests
count=0
# Initialize an empty list to store the data
data_list = []

# Create the output file
output_file = "classiccars_data.csv"


for url in urls:
    # Send an HTTP request to the URL
    r = requests.get(url)

    # Check if the request was successful
    if r.status_code == 200:
        successful_requests += 1  # Increment the successful request counter

        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')

        # Extract vehicle attributes
        attributes = {
            'URL': url,
            'Description': '',
            'Price': '',
            'VIN': '',
            'Make': '',
            'Model': '',
            'Trim': '',
            'Year': '',
            'Engine Size': '',
            'Transmission': '',
            'Exterior Color': '',
            'Interior Color': '',
            'Odometer': '',
            'Condition':''
        }

        # Find the div containing vehicle details
        cardiv = soup.find('div', class_='vehicle-description pad-sm bg-lt-gray h-card')

        if cardiv:
            # Extract vehicle description
            attributes['Description'] = ' '.join([paragraph.get_text(strip=True) for paragraph in cardiv.find_all('p')])


        try:
          cardet = soup.find('ul', class_='panel-mod pm-details-list no-border font-hostile-takeover')
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Price']=cardet.find('li',class_='border-btm p-price').find("span",class_="w50 d-inline-block b fs-18 red").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['VIN'] = cardet.find('li', class_='border-btm p-vin').find("span", class_="w40 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Make'] = cardet.find('li', class_='border-btm p-manufacturer').find("span", class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Model'] = cardet.find('li', class_='border-btm p-model').find("span", class_="w40 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Year'] = cardet.find('li',class_='border-btm dt-start').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Trim'] = cardet.find('li',class_='border-btm p-bodyType').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Engine Size'] = cardet.find('li',class_='border-btm p-engine').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Transmission'] = cardet.find('li',class_='border-btm p-transmission').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Exterior Color'] = cardet.find('li', class_='border-btm p-color').find("span", class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Interior Color']=cardet.find('span', string='Interior Color:').find_parent('li').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Odometer']=cardet.find('li',class_='border-btm p-odometer').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1
        try:
          attributes['Condition']=cardet.find('span', string='Exterior Condition:').find_parent('li').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1



        # Extract additional vehicle details (Modify class names accordingly)
        for key in attributes:
            if key != 'URL' and key != 'Description':
                element = soup.find('span', class_=f'vehicle-{key.lower().replace(" ", "-")}')
                if element:
                    attributes[key] = element.text

        # Append all the extracted data to the list
        data_list.append(attributes)
    else:
        failed_requests += 1  # Increment the failed request counter
        print(f"Failed to retrieve data from {url}. Status code: {r.status_code}")

# Create a DataFrame from the list
df = pd.DataFrame(data_list)


# Lets us know the # of successful and failed links and where they are saved
print(f'Successful requests: {successful_requests}')
print(f'Failed requests: {failed_requests}')
print(f'Data saved to {output_file}')


Successful requests: 34651
Failed requests: 0
Data saved to classiccars_data.csv


In [ ]:
df1=df

In [ ]:
df1 = df1.replace("None", "N/A", regex=True).dropna(how='all')

In [ ]:
df1 = df1[~df1.apply(lambda row: row.astype(str).str.contains('None').any(), axis=1)]

In [ ]:
df1

,URL,Description,Price,VIN,Make,Model,Trim,Year,Engine Size,Transmission,Exterior Color,Interior Color,Odometer,Condition
50,https://classiccars.com/listings/view/1771794/...,All vehicle trades considered including classi...,"$85,000",CE142F354725,Chevrolet,C10,Restomod,1972,V-8,Automatic,Orange and White,Orange and Black,1897,Excellent
208,https://classiccars.com/listings/view/1775767/...,2002 Maserati 128 GT Cambiocorsa CoupeLocated ...,"$16,500",ZAMBC38A020008466,Maserati,Cambiocorsa,Gran Turismo,2002,V-8,Automatic,Silver,Gray Leather,55400,Excellent
217,https://classiccars.com/listings/view/1769373/...,"ABSOLUTELY GORGEOUS, BRAND NEW, 1957 PORSCHE 3...",$0,1342558707,Porsche,356,Vintage Motorcars of CA,1957,Flat 4,Manual,Dark Blue,Beechwood,49,Mint
273,https://classiccars.com/listings/view/1773605/...,1972 Chevrolet Camaro SS396 Original rare big ...,"$69,000 (OBO)",1Q87U2N,Chevrolet,Camaro SS,SS 396,1972,V-8,Automatic,Midnight Bronze,Parchment,34740,Excellent
423,https://classiccars.com/listings/view/1772225/...,"Fantastic 1972 Porsche 914 Targa for sale, pow...","$23,000",4722902229,Porsche,914,Targa,1972,Flat 4,Manual,Saturn Yellow,Black,91883,Excellent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8547,https://classiccars.com/listings/view/1773378/...,1953 Chevy Panel Truck(355 SBC/3-speed auto tr...,"$38,000 (OBO)",2HPJ19765,Chevrolet,Panel Truck,Custom,1953,V-8,Automatic,Cream,Tan,865,Very Good
8778,https://classiccars.com/listings/view/1770436/...,"1937 Ford Tudor Chopped(Custom, All-Steel, 355...","$38,000 (OBO)",SW41552PA,Ford,2-Dr Sedan,Custom,1937,V-8,Automatic,Red,Gray,3634,Very Good
8857,https://classiccars.com/listings/view/1773043/...,A 1968 Porsche 912 Karmann coupe. Currently bl...,"$12,950",12803737,Porsche,912,Coupe,1968,V-4,Manual,Irish Green,Tan,35451,Poor
8980,https://classiccars.com/listings/view/1777386/...,"911 3.6 Turbo \u201cS\u201c Flachbau, Guards r...",Contact Seller,WP0AC2968RS480417,Porsche,911 Turbo S,Turbo \u201cS\u201c Flachbau,1994,Flat 6,Manual,Guards Red,Black,4900,Excellent


In [ ]:
df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

In [ ]:
df1 = df1.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

In [ ]:
# Save the DataFrame to an Excel file
output_file = 'vehicle_dataCleaned.xlsx'
df1.to_excel(output_file, index=False)

In [ ]:
# Save the DataFrame to an Excel file
output_file = 'vehicle_data.xlsx'
df.to_excel(output_file, index=False)

In [ ]:
#ARCHIVED DATA

In [ ]:
# Gives us an example of one of the many cars and their information

        cardet = soup.find('ul', class_='panel-mod pm-details-list no-border font-hostile-takeover')
        print(cardet.find('li',class_='border-btm p-price').find("span",class_="w50 d-inline-block b fs-18 red").get_text(strip=True))
        #cardet.find('li',class_='border-btm p-address').find("span",class_="w40 d-inline-block b fs-14 gray").get_text(strip=True)
        print(cardet.find('li',class_='border-btm dt-start').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True))
        print(cardet.find('li',class_='border-btm p-manufacturer').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True))
        print(cardet.find('li',class_='border-btm p-model').find("span",class_="w40 d-inline-block b fs-14 gray").get_text(strip=True))
        print(cardet.find('li',class_='border-btm p-color').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True))
        print(cardet.find('span', string='Interior Color:').find_parent('li').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True))
        print(cardet.find('li',class_='border-btm p-transmission').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True))
        # cardet.find('li',class_='border-btm p-engine').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        print(cardet.find('li',class_='border-btm p-odometer').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True))
       # cardet.find('li',class_='border-btm p-stockNumber').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
        #cardet.find('li',class_='border-btm p-vin').find("span",class_="w40 d-inline-block b fs-14 gray").get_text(strip=True)

$8,500
1994
Ford
F150
Green
Tan
Automatic
101991


In [ ]:
print(cardet)

<ul class="panel-mod pm-details-list no-border font-hostile-takeover">
<li class="border-btm p-name">
<span class="w100 font-hostile-takeover orange b fs-32rw" style="font-size: 1.25rem;">1994 Ford F150</span>
<div class="w100 d-sm-inline-block b fs-14 mrg-r-md"><div style="width:100%; min-height: 14px;"></div></div>
</li>
<li class="border-btm p-productID">
<span class="w40 d-inline-block b fs-14 mrg-r-md">Listing ID:</span>
<span class="w50 d-inline-block b fs-14 gray">CC-1781753</span>
<div class="clearfix"></div>
<div class="w40 d-inline-block b fs-14 mrg-r-md">
<div style="width:100%; min-height: 14px;"></div>
</div>
<div class="d-inline-block b fs-18">
<div id="proper-ad-classiccars_vehicledetails_inspection_contextual" style="width:100%; min-height: 14px;">
<script>
					propertag.cmd.push(function () { proper_display('classiccars_vehicledetails_inspection_contextual'); });
				</script>
</div>
</div>
</li>
<li class="border-btm p-price">
<span class="w40 d-inline-block b fs-14 

In [ ]:
# Send an HTTP request to the URL
r = requests.get('https://classiccars.com/listings/view/1777071/1972-chevrolet-c10-for-sale-in-st-louis-missouri-63114')

# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')



# Find the div Car description
cardiv = soup.find('div', class_='vehicle-description pad-sm bg-lt-gray h-card')
desc = cardiv.find_all('p')
print(desc)


# Extract and print only the text without tags and brackets
for paragraph in desc:
    print(paragraph.get_text(strip=True))

cardet = soup.find('ul', class_='panel-mod pm-details-list no-border font-hostile-takeover')


[<p><strong>NOW LIVE</strong> on Bringatrailer.com:<br/>
LOT #124389</p>, <p><em><strong>COPY/PASTE LINK BELOW TO VIEW LISTING AND BID NOW!!</strong></em><br/>
https://bringatrailer.com/listing/1972-chevrolet-c10-pickup-146/</p>, <p>This 1972 Chevrolet C10 is a Fleetside pickup that was refurbished under prior ownership and is finished in orange and white over black vinyl and orange houndstooth cloth upholstery. Power is supplied by a 350ci V8 linked with a TH350 three-speed automatic transmission and a 12-bolt rear axle. Additional equipment includes Cheyenne Super badging, a wood-lined bed floor, air conditioning, power steering, power front disc brakes, 15″ Rally-style wheels, woodgrain trim accents, and a push-button AM/FM radio. This C10 was acquired by the selling dealer in June 2023 and is now offered in Missouri with a clean Kentucky title.</p>]
NOW LIVEon Bringatrailer.com:LOT #124389
COPY/PASTE LINK BELOW TO VIEW LISTING AND BID NOW!!https://bringatrailer.com/listing/1972-che

In [ ]:
# possibly remove the nas from the excel sheet

In [ ]:
# maybe reomve condition as a column
# also remove trim

In [ ]:
# possibly better
import requests
from bs4 import BeautifulSoup
import pandas as pd


# Initialize counters
successful_requests = 0  # To count successful requests
failed_requests = 0  # To count failed requests
count=0
# Initialize an empty list to store the data
data_list = []

# Create the output file
output_file = "classiccars_data.csv"


for url in urls:
    # Send an HTTP request to the URL
    r = requests.get(url)

    # Check if the request was successful
    if r.status_code == 200:
        successful_requests += 1  # Increment the successful request counter

        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')

        # Extract vehicle attributes
        attributes = {
            'URL': url,
            'Description': '',
            'Price': '',
            'VIN': '',
            'Make': '',
            'Model': '',
            'Trim': '',
            'Year': '',
            'Engine Size': '',
            'Transmission': '',
            'Exterior Color': '',
            'Interior Color': '',
            'Odometer': '',
            'Condition':''
        }

        # Find the div containing vehicle details
        cardiv = soup.find('div', class_='vehicle-description pad-sm bg-lt-gray h-card')

        if cardiv:
            # Extract vehicle description
            attributes['Description'] = ' '.join([paragraph.get_text(strip=True) for paragraph in cardiv.find_all('p')])


        try:
          cardet = soup.find('ul', class_='panel-mod pm-details-list no-border font-hostile-takeover')

         # Extract and assign values to attributes
          attributes['Price']=cardet.find('li',class_='border-btm p-price').find("span",class_="w50 d-inline-block b fs-18 red").get_text(strip=True)
          attributes['VIN'] = cardet.find('li', class_='border-btm p-vin').find("span", class_="w40 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Make'] = cardet.find('li', class_='border-btm p-manufacturer').find("span", class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Model'] = cardet.find('li', class_='border-btm p-model').find("span", class_="w40 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Year'] = cardet.find('li',class_='border-btm dt-start').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Trim'] = cardet.find('li',class_='border-btm p-bodyType').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Engine Size'] = cardet.find('li',class_='border-btm p-engine').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Transmission'] = cardet.find('li',class_='border-btm p-transmission').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Exterior Color'] = cardet.find('li', class_='border-btm p-color').find("span", class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Interior Color']=cardet.find('span', string='Interior Color:').find_parent('li').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Odometer']=cardet.find('li',class_='border-btm p-odometer').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)
          attributes['Condition']=cardet.find('span', string='Exterior Condition:').find_parent('li').find("span",class_="w50 d-inline-block b fs-14 gray").get_text(strip=True)

        # Do something with the extracted data, e.g., print or store it
        except AttributeError as e:
          # Handle the exception (e.g., print an error message or set default values)
          #print(f"Warning: {e}")
          count+=1


        # Extract additional vehicle details (Modify class names accordingly)
        for key in attributes:
            if key != 'URL' and key != 'Description':
                element = soup.find('span', class_=f'vehicle-{key.lower().replace(" ", "-")}')
                if element:
                    attributes[key] = element.text

        # Append all the extracted data to the list
        data_list.append(attributes)
    else:
        failed_requests += 1  # Increment the failed request counter
        print(f"Failed to retrieve data from {url}. Status code: {r.status_code}")

# Create a DataFrame from the list
df = pd.DataFrame(data_list)



print(f'Successful requests: {successful_requests}')
print(f'Failed requests: {failed_requests}')
print(f'Data saved to {output_file}')


Successful requests: 1730
Failed requests: 0
Data saved to classiccars_data.csv
